In [1]:
import torch
import torch.nn as nn
from typing import List

class Net(nn.Module):
    def __init__(self, in_dim: int=2, out_dim: int=2, h_dims: List[int]=[512]*4) -> None:
        super().__init__()        
        ins = [in_dim] + h_dims
        outs = h_dims + [out_dim]
        self.layers = nn.ModuleList(
            [nn.Sequential(nn.Linear(in_d, out_d), nn.LeakyReLU()) for in_d, out_d in zip(ins, outs)]
            )
        self.out = nn.Sequential(nn.Linear(out_dim, out_dim))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x)
        return self.out(x)
    
net=Net(1,1,[64]*4)
x_train = torch.rand((1000,1))
y_train = torch.sin(x_train*2*3.14159)
x_val = torch.rand((100,1))
y_val = torch.sin(x_val*2*3.14159)
train_data = torch.cat((x_train, y_train), dim=1)
val_data = torch.cat((x_val, y_val), dim=1)

In [2]:
from typing import Any
from torch.nn.modules import Module
from foxutils.trainer import EMATrainer as Trainer
import time

class MyTrainer(Trainer):
    
    def train_step(self, model: Module, batch: Any, batch_idx: int):
        inputs=batch[:,:1]
        targets=batch[:,1:]
        time.sleep(0.01)
        return torch.nn.functional.mse_loss(model(inputs), targets)
    
    def validation_step(self, model: Module, batch: Any, batch_idx: int):
        inputs=batch[:,:1]
        targets=batch[:,1:]
        time.sleep(0.1)
        return torch.nn.functional.mse_loss(model(inputs), targets)
    
trainer=MyTrainer()
trainer.info_available_configs(print_info=True)

Mandatory Configuration:
    batch_size_train (int): Batch size for training.
    lr (float): Initial learning rate.
    num_epochs (int): Number of epochs for training.
    project_path (str): Path to save the training results.

Optional Configuration:
    batch_size_val (int): Batch size for validation. Default is the same as batch_size_train. If not set, the batch size will be the same as the training batch size.
    checkpoint_save_frequency (int): Frequency of saving checkpoints. The unit is epoch. If not set, the checkpoint will be saved every num_epochs//10 epoch.
    device_type (str, possible options: ['cpu', 'cuda', 'mps', 'gpu', 'tpu', 'auto'], default value: auto): The hardware to run on. Will pass to `accelerator` in `Fabric`.
    do_ema_validation (bool, default value: True): Whether to perform additional validation after EMA weight update
    ema_coef (float, default value: 0.9): Exponential moving average coefficient for EMA weights recording
    ema_weights_update_freq

'Mandatory Configuration:\n    batch_size_train (int): Batch size for training.\n    lr (float): Initial learning rate.\n    num_epochs (int): Number of epochs for training.\n    project_path (str): Path to save the training results.\n\nOptional Configuration:\n    batch_size_val (int): Batch size for validation. Default is the same as batch_size_train. If not set, the batch size will be the same as the training batch size.\n    checkpoint_save_frequency (int): Frequency of saving checkpoints. The unit is epoch. If not set, the checkpoint will be saved every num_epochs//10 epoch.\n    device_type (str, possible options: [\'cpu\', \'cuda\', \'mps\', \'gpu\', \'tpu\', \'auto\'], default value: auto): The hardware to run on. Will pass to `accelerator` in `Fabric`.\n    do_ema_validation (bool, default value: True): Whether to perform additional validation after EMA weight update\n    ema_coef (float, default value: 0.9): Exponential moving average coefficient for EMA weights recording\n  

In [3]:
trainer.train(net, train_data, val_data,
              project_path='./learn_sin',
              num_epochs=100,
              lr=1e-3,
              batch_size_train=32,
              run_name='test',
              show_iteration_bar=False,
              checkpoint_save_frequency=20,
              do_ema_validation=False,
              log_iteration_loss=False,
              lr_scheduler="constant",
              optimizer="SGD",
              precision="64",
              random_seed=42,
              save_model_structure=False,
              ema_weights_update_freq=5,
              version="my",
              warmup_epoch=50,
              )

You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Finished Epoch:  40%|####      | 40/100 [00:00<?, ?epoch/s]

2024-09-11 14:38:13.478758: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 14:38:13.490396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 14:38:13.504468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 14:38:13.508452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 14:38:13.518867: I tensorflow/core/platform/cpu_feature_guar

In [2]:
config_mixin = TrainConfigMixin()
config_mixin.register_configs()

In [3]:
config_mixin.info_available_configs(print_info=True)

Mandatory Configuration:
    batch_size_train (int, group: Control): Batch size for training.
    lr (float, group: Control): Initial learning rate.
    num_epochs (int, group: Control): Number of epochs for training.
    project_path (str, group: ProjectInfo): Path to save the training results.

Optional Configuration:
    batch_size_val (int, group: Control): Batch size for validation. Default is the same as batch_size_train. If not set, the batch size will be the same as the training batch size.
    checkpoint_save_frequency (int, group: Control): Frequency of saving checkpoints. The unit is epoch. If not set, the checkpoint will be saved every num_epochs//10 epoch.
    device_type (str, possible options: ['cpu', 'cuda', 'mps', 'gpu', 'tpu', 'auto'], default value: auto, group: Control): The hardware to run on. Will pass to `accelerator` in `Fabric`.
    final_lr (float, group: Control): Final learning rate for lr_scheduler. If not set, the final learning rate will be the same as th

'Mandatory Configuration:\n    batch_size_train (int, group: Control): Batch size for training.\n    lr (float, group: Control): Initial learning rate.\n    num_epochs (int, group: Control): Number of epochs for training.\n    project_path (str, group: ProjectInfo): Path to save the training results.\n\nOptional Configuration:\n    batch_size_val (int, group: Control): Batch size for validation. Default is the same as batch_size_train. If not set, the batch size will be the same as the training batch size.\n    checkpoint_save_frequency (int, group: Control): Frequency of saving checkpoints. The unit is epoch. If not set, the checkpoint will be saved every num_epochs//10 epoch.\n    device_type (str, possible options: [\'cpu\', \'cuda\', \'mps\', \'gpu\', \'tpu\', \'auto\'], default value: auto, group: Control): The hardware to run on. Will pass to `accelerator` in `Fabric`.\n    final_lr (float, group: Control): Final learning rate for lr_scheduler. If not set, the final learning rate

In [4]:
config_mixin.set_config_items(project_path='./learn_sin',
                              batch_size_train=32,
                              lr=1e-3,
                              num_epochs=1000,
                              final_lr=1e-6,)

In [7]:
config_mixin.save_configs_to_yaml('./control')

In [19]:
class A:
    
    _my_param2=None
    
    def change_param(self):
            self._my_param2={"a":1}
a=A()
a2=A()
a2.change_param()
print(a._my_param2)
print(a2._my_param2)

None
{'a': 1}


In [9]:
a = {'a':1}
b = a.copy()

b['a']=2

a

{'a': 1}